In [1]:
#importing necessary modules
import os
import numpy as np
import pandas as pd
import sys
print(sys.version)
print("numpy", np.__version__)
print("pandas", pd.__version__)

3.11.6 (main, Jun  7 2024, 07:09:59) [GCC 13.2.0]
numpy 1.26.4
pandas 2.2.2


In [2]:
chromosomes = ['chr' + str(i) for i in np.arange(1, 20, 1)] + ["chrX", "chrY"]
chromosomes

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chrX',
 'chrY']

In [3]:
chromosomes_name_table = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Navnit/genomes/mouse/Chromosome_naming_table_17.09.2024_Vakil.tsv"
chromosomes_name_table = pd.read_csv(chromosomes_name_table, sep = "\t", header = 0)
chromosomes_name_table = chromosomes_name_table[chromosomes_name_table["UCSC style name"].isin(chromosomes)]
chromosomes_name_table = chromosomes_name_table.loc[:, ["RefSeq seq accession", "UCSC style name"]]
chromosomes_name_table = chromosomes_name_table.rename(columns = {"UCSC style name" : "chrom"})
chromosomes_name_table

,RefSeq seq accession,chrom
0,NC_000067.7,chr1
1,NC_000068.8,chr2
2,NC_000069.7,chr3
3,NC_000070.7,chr4
4,NC_000071.7,chr5
5,NC_000072.7,chr6
6,NC_000073.7,chr7
7,NC_000074.7,chr8
8,NC_000075.7,chr9
9,NC_000076.7,chr10


In [4]:
path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/UCSC_Table_Browser/knownGene_GRCm39_GENCODE.VM36.tsv"
allGenecode = pd.read_csv(path, sep = "\t")
allGenecode = allGenecode.loc[:, ["name", "#chrom", "strand", "chromStart", "chromEnd"]]
allGenecode = allGenecode.rename(columns = {"#chrom" : "chrom"})
allGenecode

,name,chrom,strand,chromStart,chromEnd
0,ENSMUST00000193812.2,chr1,+,3143475,3144545
1,ENSMUST00000082908.3,chr1,+,3172238,3172348
2,ENSMUST00000162897.2,chr1,-,3276123,3286567
3,ENSMUST00000159265.2,chr1,-,3276745,3285855
4,ENSMUST00000070533.5,chr1,-,3284704,3741721
...,...,...,...,...,...
278370,ENSMUST00000180208.2,chr5_JH584299v1_random,-,910288,913083
278371,ENSMUST00000178327.2,chr5_JH584299v1_random,+,921941,924675
278372,ENSMUST00000177783.2,chr7_GL456219v1_random,-,94185,95088
278373,ENSMUST00000177979.2,chr7_GL456219v1_random,-,149260,150163


In [5]:
path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/UCSC_Table_Browser/knownCanonical_Genes_GRCm39_GENCODE.VM36.tsv"
canonGenecode = pd.read_csv(path, sep = "\t")
canonGenecode["Gene ID"] = canonGenecode["protein"].str.split(".").str[0]
#
canonGenecode = canonGenecode[canonGenecode["#chrom"].isin(chromosomes)]
#
canonGenecode = canonGenecode.loc[:, ["transcript", "Gene ID"]]
canonGenecode = canonGenecode.rename(columns = {"transcript" : "name"})
print(canonGenecode.shape)

canonGenecode = pd.merge(canonGenecode, allGenecode, on = "name", how = "left")
canonGenecode = pd.merge(chromosomes_name_table, canonGenecode, on = "chrom")
canonGenecode = canonGenecode.loc[:, ["RefSeq seq accession", "chromStart", "chromEnd", "Gene ID", "name", "strand"]]
canonGenecode = canonGenecode.rename(columns = {"RefSeq seq accession" : "chrom",
                                                 "chromStart" : "txStart",
                                                 "chromEnd" : "txEnd",
                                                 "name" : "#name"})

canonGenecode

(78202, 2)


,chrom,txStart,txEnd,Gene ID,#name,strand
0,NC_000067.7,165957806,165994079,ENSMUSG00000000544,ENSMUST00000060833.14,+
1,NC_000067.7,161608257,161616064,ENSMUSG00000000817,ENSMUST00000000834.4,-
2,NC_000067.7,36550947,36567318,ENSMUSG00000001138,ENSMUST00000001166.14,+
3,NC_000067.7,36461145,36484352,ENSMUSG00000001143,ENSMUST00000125304.8,-
4,NC_000067.7,186453174,186481555,ENSMUSG00000001305,ENSMUST00000001339.6,-
...,...,...,...,...,...,...
78197,NC_000087.8,85425286,85425394,ENSMUSG00002076147,ENSMUST00020182676.1,-
78198,NC_000087.8,83823856,83823964,ENSMUSG00002076259,ENSMUST00020182612.1,-
78199,NC_000087.8,86337285,86337393,ENSMUSG00002076549,ENSMUST00020181692.1,-
78200,NC_000087.8,54925575,54925683,ENSMUSG00002076770,ENSMUST00020183724.1,-


In [6]:
canonGenecode["#name"].value_counts()

#name
ENSMUST00000060833.14    1
ENSMUST00000246328.1     1
ENSMUST00000362961.1     1
ENSMUST00000247074.1     1
ENSMUST00000318361.1     1
                        ..
ENSMUST00000204200.2     1
ENSMUST00000164674.3     1
ENSMUST00000204643.2     1
ENSMUST00000203812.2     1
ENSMUST00020183589.1     1
Name: count, Length: 78202, dtype: int64

In [7]:
canonGenecode["Gene ID"].value_counts() 

Gene ID
ENSMUSG00000000544    1
ENSMUSG00000120893    1
ENSMUSG00000121126    1
ENSMUSG00000121073    1
ENSMUSG00000121052    1
                     ..
ENSMUSG00000107823    1
ENSMUSG00000107821    1
ENSMUSG00000107819    1
ENSMUSG00000107818    1
ENSMUSG00002076920    1
Name: count, Length: 78202, dtype: int64

In [8]:
P_drive_path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/Genes_Promoters_CpG_islands_for_Tae/"

canonGenecode.to_csv(P_drive_path + "knownGenes_canononical_GRCm39_GENCODE.VM36.bed", sep = "\t", index = False, header = False)


In [9]:
P_drive_path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/Genes_Promoters_CpG_islands_for_Tae/"

beyond = 2000

#upstream TSS region
DF_upstream_bounds = pd.DataFrame({})
for strand in ["-", "+"]:
    tmp = canonGenecode[canonGenecode["strand"] == strand].copy()
    if strand == "+":
        tmp.loc[:, "End"] = tmp["txStart"]
        tmp.loc[:, "Start"] = tmp["txStart"] - beyond
    if strand == "-":
        tmp.loc[:, "Start"] = tmp["txEnd"]
        tmp.loc[:, "End"] = tmp["txEnd"] + beyond
        
    tmp = tmp.loc[:, ["chrom", "Start", "End", "Gene ID", "#name", "strand"]]
    DF_upstream_bounds = pd.concat([DF_upstream_bounds, tmp])
    
DF_upstream_bounds.to_csv(P_drive_path + "knownGenes_canonTr_"+str(beyond)+"bp_upstream_TSS_feature_GRCm39_GENCODE.VM36.bed", 
                          header = False, index = False, sep = '\t')

In [10]:
path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/UCSC_Table_Browser/CpG_islands_GRCm39.tsv"
CpG_islands = pd.read_csv(path, sep = "\t")
CpG_islands = CpG_islands.loc[:, ["chrom", "chromStart", "chromEnd"]]
CpG_islands = pd.merge(chromosomes_name_table, CpG_islands, on = "chrom")
CpG_islands = CpG_islands.loc[:, ["RefSeq seq accession", "chromStart", "chromEnd"]]

P_drive_path = "/nfs/nas12.ethz.ch/fs1201/green_groups_let_public/Euler/Vakil/mouse_genome_annotation/Genes_Promoters_CpG_islands_for_Tae/"
CpG_islands.to_csv(P_drive_path + "allCpG_islands_GRCm39.bed", 
                          header = False, index = False, sep = '\t')
CpG_islands

,RefSeq seq accession,chromStart,chromEnd
0,NC_000067.7,91225617,91226516
1,NC_000067.7,38665874,38667165
2,NC_000067.7,57016263,57016551
3,NC_000067.7,85721048,85722212
4,NC_000067.7,88997698,88998967
...,...,...,...
15953,NC_000087.8,90795245,90797512
15954,NC_000087.8,90804558,90805088
15955,NC_000087.8,90829598,90829834
15956,NC_000087.8,90835535,90835771
